In [66]:
# Import libraries

import numpy as np
import pandas as pd
import locale
from locale import atof
from sklearn.preprocessing import LabelEncoder
import datetime

# matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [67]:
# Import dataset

dataset = '2013-2018.csv'
df_raw = pd.read_csv(dataset,sep=';',low_memory=False)

In [68]:
# Initially check raw data

df_raw.describe()
#df_raw.head()
#df_raw.tail()

,BOROUGH\n,BLOCK\n,LOT\n,ZIP\n,LAND SQUARE FEET\n,GROSS SQUARE FEET\n,YEAR BUILT\n,TAX CLASS AT TIME OF SALE\n,SALE PRICE\n
count,532355.000000,532355.000000,532355.000000,532353.000000,5.323010e+05,5.323020e+05,532352.000000,532355.000000,5.323550e+05
mean,2.905274,4059.324622,377.697589,10764.631553,3.190654e+03,4.009219e+03,1763.856125,1.746166,1.092045e+06
std,1.318935,3524.042159,657.727629,981.241821,5.230171e+04,3.296691e+04,572.733362,0.910443,1.422370e+07
min,1.000000,1.000000,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,1.000000,0.000000e+00
25%,1.000000,1223.000000,22.000000,10128.000000,0.000000e+00,0.000000e+00,1920.000000,1.000000,0.000000e+00
50%,3.000000,2999.000000,50.000000,11207.000000,1.800000e+03,1.144000e+03,1935.000000,2.000000,3.500000e+05
75%,4.000000,6020.000000,1001.000000,11355.000000,2.800000e+03,2.237000e+03,1965.000000,2.000000,7.500000e+05
max,5.000000,16350.000000,9139.000000,11697.000000,2.930553e+07,8.942176e+06,2019.000000,4.000000,4.111112e+09


In [69]:
# Data transformation: Rename columns & extract year from sale date

df_raw.columns = df_raw.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
df_raw["year"] = pd.DatetimeIndex(df_raw["sale_date"]).year

In [70]:
df_raw

,borough,neighborhood,building_class_category,tax_class,block,lot,ease-ment,building_class,address,apartment_number,...,commercial_units,total_units,land_square_feet,gross_square_feet,year_built,tax_class_at_time_of_sale,building_class_at_time_of_sale,sale_price,sale_date,year
0,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,2907,24,NaN,A1,4090 PARK AVENUE,NaN,...,0,1,2500.0,1474.0,1901.0,1,A1,241500,17.08.18,2018
1,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3030,62,NaN,A1,4463 PARK AVENUE,NaN,...,0,1,1578.0,1470.0,1899.0,1,A1,180000,22.03.18,2018
2,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3030,62,NaN,A1,4463 PARK AVENUE,NaN,...,0,1,1578.0,1470.0,1899.0,1,A1,455000,28.11.18,2018
3,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3030,70,NaN,A1,4445 PARK AVENUE,NaN,...,0,1,1694.0,1497.0,1899.0,1,A1,255000,05.03.18,2018
4,2,BATHGATE,01 ONE FAMILY DWELLINGS,1,3039,28,NaN,A1,2329 WASHINGTON AVENUE,NaN,...,0,1,1103.0,1290.0,1910.0,1,A1,340000,22.02.18,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532350,5,WOODROW,05 TAX CLASS 1 VACANT LAND,1B,7092,200,,V0,N/A CRABTREE AVENUE,,...,0,0,8631.0,0.0,2010.0,1,V0,0,19.09.13,2013
532351,5,WOODROW,05 TAX CLASS 1 VACANT LAND,1B,7105,574,,V0,TURNER STREET,,...,0,0,2636.0,0.0,0.0,1,V0,0,19.09.13,2013
532352,5,WOODROW,22 STORE BUILDINGS,4,7100,16,,K6,639 VETERANS ROAD WEST,,...,1,1,21663.0,6950.0,2008.0,4,K6,1300000,29.05.13,2013
532353,5,WOODROW,22 STORE BUILDINGS,4,7105,520,,K1,2790 ARTHUR KILL ROAD,,...,1,1,489656.0,159600.0,2010.0,4,K6,14850914,01.07.13,2013


In [71]:
# Drop empty columns ['ease-ment', 'apartment_number']

df_raw = df_raw.drop(['ease-ment', 'apartment_number'], axis = 1)

# Filter Data

#df_raw[df_raw['total_units'] > 1500] = np.nan 
df_raw[df_raw['sale_price'] < 100000] = np.nan 
df_raw[df_raw['gross_square_feet'] < 25] = np.nan 
df_raw[df_raw['tax_class'] == '4'] = np.nan             #filter for commerical
df_raw[df_raw['zip'] < 1] = np.nan 
df_raw[df_raw['zip'] == np.nan] = np.nan # hope to get rid of the nans in zip code             

#leave total_unit filter out for now
#df_raw[df_raw['total_units'] > 10] = np.nan 

df_raw = df_raw.dropna(how='all')
df_raw = df_raw.dropna(how='any') # drops all rows with nan values (41 values in the 2013-2018 dataset)
 

In [72]:
## Add necessary columns

# Add price per sq meter (ppsqm) column

df_raw['ppsqft'] = df_raw['sale_price'] / df_raw['gross_square_feet']


# create a second zip column for later

df_raw['zip_copy'] = df_raw['zip']

df_raw

,borough,neighborhood,building_class_category,tax_class,block,lot,building_class,address,zip,residential_units,...,land_square_feet,gross_square_feet,year_built,tax_class_at_time_of_sale,building_class_at_time_of_sale,sale_price,sale_date,year,ppsqft,zip_copy
0,2.0,BATHGATE,01 ONE FAMILY DWELLINGS,1,2907.0,24.0,A1,4090 PARK AVENUE,10457.0,1,...,2500.0,1474.0,1901.0,1.0,A1,241500.0,17.08.18,2018.0,163.839891,10457.0
1,2.0,BATHGATE,01 ONE FAMILY DWELLINGS,1,3030.0,62.0,A1,4463 PARK AVENUE,10457.0,1,...,1578.0,1470.0,1899.0,1.0,A1,180000.0,22.03.18,2018.0,122.448980,10457.0
2,2.0,BATHGATE,01 ONE FAMILY DWELLINGS,1,3030.0,62.0,A1,4463 PARK AVENUE,10457.0,1,...,1578.0,1470.0,1899.0,1.0,A1,455000.0,28.11.18,2018.0,309.523810,10457.0
3,2.0,BATHGATE,01 ONE FAMILY DWELLINGS,1,3030.0,70.0,A1,4445 PARK AVENUE,10457.0,1,...,1694.0,1497.0,1899.0,1.0,A1,255000.0,05.03.18,2018.0,170.340681,10457.0
4,2.0,BATHGATE,01 ONE FAMILY DWELLINGS,1,3039.0,28.0,A1,2329 WASHINGTON AVENUE,10458.0,1,...,1103.0,1290.0,1910.0,1.0,A1,340000.0,22.02.18,2018.0,263.565891,10458.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532343,5.0,WOODROW,02 TWO FAMILY DWELLINGS,1,7317.0,124.0,B2,83 ROBIN COURT,10309.0,2,...,7525.0,2160.0,1994.0,1.0,B2,520000.0,18.06.13,2013.0,240.740741,10309.0
532344,5.0,WOODROW,02 TWO FAMILY DWELLINGS,1,7317.0,143.0,B2,64 ROBIN COURT,10309.0,2,...,3380.0,2400.0,1994.0,1.0,B2,482500.0,03.10.13,2013.0,201.041667,10309.0
532346,5.0,WOODROW,02 TWO FAMILY DWELLINGS,1,7346.0,12.0,B2,476 HARGOLD AVENUE,10309.0,2,...,7933.0,3875.0,1997.0,1.0,B2,816970.0,27.12.13,2013.0,210.830968,10309.0
532347,5.0,WOODROW,02 TWO FAMILY DWELLINGS,1,7349.0,10.0,B9,63 PHEASANT LANE,10309.0,2,...,2590.0,2450.0,1998.0,1.0,B9,505000.0,15.05.13,2013.0,206.122449,10309.0


In [73]:
# Check if data makes sense, i.e. average ppsqft

df_raw[df_raw['borough'] == 1].mean()

borough                      1.000000e+00
block                        1.204852e+03
lot                          7.402310e+02
zip                          1.003102e+04
land_square_feet             6.661745e+03
gross_square_feet            1.251014e+04
year_built                   1.751868e+03
tax_class_at_time_of_sale    1.869084e+00
sale_price                   7.923806e+06
year                         2.016611e+03
ppsqft                       1.368479e+03
zip_copy                     1.003102e+04
dtype: float64

In [74]:
# indexing

df_raw = df_raw.set_index(['year','zip'])
df_raw = df_raw.sort_values(by=["year","zip"])

df_raw

borough               neighborhood  \
year   zip                                           
2013.0 10001.0      1.0  CHELSEA                     
       10001.0      1.0  CHELSEA                     
       10001.0      1.0  CHELSEA                     
       10001.0      1.0  CHELSEA                     
       10001.0      1.0  CHELSEA                     
...                 ...                        ...   
2018.0 11694.0      4.0              ROCKAWAY PARK   
       11694.0      4.0              ROCKAWAY PARK   
       11694.0      4.0              ROCKAWAY PARK   
       11694.0      4.0              ROCKAWAY PARK   
       11694.0      4.0              ROCKAWAY PARK   

                                     building_class_category tax_class  \
year   zip                                                               
2013.0 10001.0  01  ONE FAMILY DWELLINGS                             1   
       10001.0  01  ONE FAMILY DWELLINGS                             2   
       10001.0  07  RENTALS - WALKUP APARTMENTS                     2B   
       10001.0  07  RENTALS - WALKUP APARTMENTS                     2B   
       10001.0  07  RENTALS - WALKUP APARTMENTS                     2B   
...                                                      ...       ...   
2018.0 11694.0               13 CONDOS - ELEVATOR APARTMENTS         2   
       11694.0               13 CONDOS - ELEVATOR APARTMENTS         2   
       11694.0               13 CONDOS - ELEVATOR APARTMENTS         2   
       11694.0               13 CONDOS - ELEVATOR APARTMENTS         2   
       11694.0               13 CONDOS - ELEVATOR APARTMENTS         2   

                  block     lot building_class  \
year   zip                                       
2013.0 10001.0    774.0    81.0             A4   
       10001.0    775.0     5.0             C6   
       10001.0    699.0    37.0             C4   
       10001.0    753.0    28.0             C1   
       10001.0    753.0    28.0             C1   
...                 ...     ...            ...   
2018.0 11694.0  16226.0  1067.0             R4   
       11694.0  16226.0  1079.0             R4   
       11694.0  16228.0  1012.0             R4   
       11694.0  16228.0  1015.0             R4   
       11694.0  16228.0  1023.0             R4   

                                                  address residential_units  \
year   zip                                                                    
2013.0 10001.0  280 WEST 25TH   STREET                                    1   
       10001.0  267 WEST 25TH   STREET                                   31   
       10001.0  311 10TH   AVENUE                                         6   
       10001.0  315 WEST 29 STREET                                       10   
       10001.0  315 WEST 29TH STREET                                     10   
...                                                   ...               ...   
2018.0 11694.0                    116-12 OCEAN PROMENADE                  1   
       11694.0                      136 BEACH 117 STREET                  1   
       11694.0                    118-30 OCEAN PROMENADE                  1   
       11694.0                    118-20 OCEAN PROMENADE                  1   
       11694.0                    118-20 OCEAN PROMENADE                  1   

               commercial_units total_units  land_square_feet  \
year   zip                                                      
2013.0 10001.0                0           1             937.0   
       10001.0                1          32            9505.0   
       10001.0                1           7            2467.0   
       10001.0                0          10            2173.0   
       10001.0                0          10            2173.0   
...                         ...         ...               ...   
2018.0 11694.0                0           1               0.0   
       11694.0                0           1               0.0   
       11694.0       

In [75]:
df_trad = pd.DataFrame(df_raw)
df_trad = df_trad.drop(['block','lot','sale_date','sale_price','neighborhood','tax_class','tax_class_at_time_of_sale','building_class_category','building_class','address','building_class_at_time_of_sale','zip_copy'],axis=1)

df_trad.land_square_feet[df_trad['land_square_feet']==0] = np.nan
df_trad['land_square_feet'] = df_trad.land_square_feet.fillna(method='ffill')

df_trad.gross_square_feet[df_trad['gross_square_feet']==0] = np.nan
df_trad['gross_square_feet'] = df_trad.gross_square_feet.fillna(method='ffill') 

df_trad.year_built[df_trad['year_built']==0] = np.nan
df_trad['year_built'] = df_trad.year_built.fillna(method='ffill') 

df_trad = df_trad.mean(level=[0,1])
df_trad = df_trad.sort_values(by=["year", "zip"], ascending = [False, True])

df_trad

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda/envs/Python3/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda/envs/Python3/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


borough  land_square_feet  gross_square_feet   year_built  \
year   zip                                                                  
2018.0 10001.0      1.0      11737.470000        1640.550000  1987.020000   
       10002.0      1.0      10914.616822        1782.018692  1986.037383   
       10003.0      1.0      40063.034884        2927.680233  1948.069767   
       10004.0      1.0       3680.000000        1013.500000  1978.770833   
       10005.0      1.0       3680.000000        1002.627119  1959.152542   
...                 ...               ...                ...          ...   
2013.0 11436.0      4.0       2788.762791        1427.986047  1933.241860   
       11691.0      4.0       5112.708609        2970.278146  1963.039735   
       11692.0      4.0       2989.357143        2529.071429  1996.914286   
       11693.0      4.0       3529.263158        1774.263158  1933.763158   
       11694.0      4.0       4809.928571        2394.182540  1940.341270   

                     ppsqft  
year   zip                   
2018.0 10001.0  1830.423318  
       10002.0  1640.060071  
       10003.0  1830.758578  
       10004.0  1519.703220  
       10005.0  1290.443401  
...                     ...  
2013.0 11436.0   220.466667  
       11691.0   178.396298  
       11692.0   201.260163  
       11693.0   171.277672  
       11694.0   256.893480  

[1045 rows x 5 columns]

In [12]:
#################################################################################################################
df_trad.to_csv(r'dataframe_2013-2018_tradreg.csv')
#################################################################################################################

In [76]:
# average ppsqft per zip code and year

df = pd.DataFrame(df_raw[['ppsqft']])
df['sales_per_zip'] = df.groupby(level=[0,1]).agg(['size'])
df = df.mean(level=[0,1])
df = df.sort_values(by=["year", "zip"], ascending = [False, True])

df


ppsqft  sales_per_zip
year   zip                                
2018.0 10001.0  1830.423318            100
       10002.0  1640.060071            214
       10003.0  1830.758578            172
       10004.0  1519.703220             48
       10005.0  1290.443401            118
...                     ...            ...
2013.0 11436.0   220.466667            215
       11691.0   178.396298            151
       11692.0   201.260163             70
       11693.0   171.277672             38
       11694.0   256.893480            126

[1045 rows x 2 columns]

In [77]:
# extract boroughs from raw_df

borough = pd.DataFrame(df_raw['borough'])
borough = borough.droplevel(0)
borough = borough[~borough.index.get_level_values(0).duplicated()]
borough = borough.sort_values(by=["zip"], ascending = True)
borough.rename(columns={'borough':'borough_no'}, inplace = True)

borough.tail(5)

,borough_no
zip,
11436.0,4.0
11691.0,4.0
11692.0,4.0
11693.0,4.0
11694.0,4.0


In [78]:
# Merge/add borough column to dataframe 
df2 = df.join(borough)

df2


#df = pd.merge(df, borough, on = ["year", "zip"], how = "left")
#old merge function causing problems

ppsqft  sales_per_zip  borough_no
year   zip                                            
2018.0 10001.0  1830.423318            100         1.0
       10002.0  1640.060071            214         1.0
       10003.0  1830.758578            172         1.0
       10004.0  1519.703220             48         1.0
       10005.0  1290.443401            118         1.0
...                     ...            ...         ...
2013.0 11436.0   220.466667            215         4.0
       11691.0   178.396298            151         4.0
       11692.0   201.260163             70         4.0
       11693.0   171.277672             38         4.0
       11694.0   256.893480            126         4.0

[1045 rows x 3 columns]

In [79]:
# Import csv cafe_licenses_applicaton data

cafe_licenses_applicaton = 'cafe_licenses_applications_2013_2018.csv'
cafes = pd.read_csv(cafe_licenses_applicaton,sep=',',low_memory=False)

cafes.columns = cafes.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
cafes.rename(columns={'year_of_start_date':'year'}, inplace = True)
cafes = cafes.set_index(['year','zip'])
cafes = cafes.sort_values(by=["year","zip"])


# cafes dataset for renewalss
cafes_renewal = cafes[cafes["application_or_renewal"] == "Renewal"]
cafes_renewal.rename(columns={'distinct_count_of_application_id':'renewals_applications', 'distinct_count_of_license_number':'renewals_licenses'}, inplace = True)
cafes_renewal = cafes_renewal.drop("application_or_renewal", axis=1)
cafes_renewal['cumsum_application_renewal'] = cafes_renewal['renewals_applications']
cafes_renewal['cumsum_application_renewal'] = cafes_renewal.groupby(level=[1]).cumsum()


# cafes dataset for applications
cafes_application = cafes[cafes["application_or_renewal"] == "Application"]
cafes_application.rename(columns={'distinct_count_of_application_id':'application_applications', 'distinct_count_of_license_number':'application_licenses'}, inplace = True)
cafes_application = cafes_application.drop("application_or_renewal", axis=1)
cafes_application['applications'] = (cafes_application['application_applications'])
cafes_application['cumsum_application'] = cafes_application.applications.groupby(level=[1]).cumsum()

cafes_application



application_applications  applications  cumsum_application
year   zip                                                                
2013.0 10001.0                      17.0          17.0                17.0
       10002.0                      19.0          19.0                19.0
       10003.0                      12.0          12.0                12.0
       10004.0                       2.0           2.0                 2.0
       10005.0                       4.0           4.0                 4.0
...                                  ...           ...                 ...
2018.0 11691.0                      50.0          50.0               221.0
       11692.0                      19.0          19.0                63.0
       11693.0                      21.0          21.0                87.0
       11694.0                      30.0          30.0               172.0
       11697.0                       2.0           2.0                14.0

[1875 rows x 3 columns]

In [80]:
# Merge datasets cafe and original data

#cafes_application = cafes_application.join(cafes_renewal)

df3 = df2.join(cafes_application)
df3

#df2 = pd.join(df,cafes_renewal, on = ['year','zip'], how= "left")
#old merge function causing problems

ppsqft  sales_per_zip  borough_no  \
year   zip                                               
2018.0 10001.0  1830.423318            100         1.0   
       10002.0  1640.060071            214         1.0   
       10003.0  1830.758578            172         1.0   
       10004.0  1519.703220             48         1.0   
       10005.0  1290.443401            118         1.0   
...                     ...            ...         ...   
2013.0 11436.0   220.466667            215         4.0   
       11691.0   178.396298            151         4.0   
       11692.0   201.260163             70         4.0   
       11693.0   171.277672             38         4.0   
       11694.0   256.893480            126         4.0   

                application_applications  applications  cumsum_application  
year   zip                                                                  
2018.0 10001.0                     158.0         158.0               708.0  
       10002.0                     132.0         132.0               622.0  
       10003.0                     114.0         114.0               515.0  
       10004.0                      35.0          35.0               120.0  
       10005.0                      25.0          25.0                77.0  
...                                  ...           ...                 ...  
2013.0 11436.0                       4.0           4.0                 4.0  
       11691.0                       8.0           8.0                 8.0  
       11692.0                       1.0           1.0                 1.0  
       11693.0                       4.0           4.0                 4.0  
       11694.0                       3.0           3.0                 3.0  

[1045 rows x 6 columns]

In [81]:
########################### DELETE ###########################

# Import csv Population data

population_2010_2018 = '2010-2018_Population_.csv'
population = pd.read_csv(population_2010_2018,sep=',',low_memory=False)

population.columns = population.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
population.rename(columns={'zip_codes':'zip'}, inplace = True)
population = population.set_index(['year','zip'])
population = population.sort_values(by=["year","zip"])

population
########################### DELETE ###########################

borough         neighborhood  population
year zip                                              
2010 10001  Manhattan  Chelsea and Clinton    36892.09
     10002  Manhattan      Lower East Side    36892.09
     10003  Manhattan      Lower East Side    36892.09
     10004  Manhattan      Lower Manhattan    36892.09
     10005  Manhattan      Lower Manhattan    36892.09
...               ...                  ...         ...
2018 11692     Queens            Rockaways    37359.00
     11693     Queens            Rockaways    37359.00
     11694     Queens            Rockaways    37359.00
     11695     Queens            Rockaways    37359.00
     11697     Queens            Rockaways    37359.00

[1602 rows x 3 columns]

In [82]:
# Join population data to dataset

df4 = df3.join(population)
df4

ppsqft  sales_per_zip  borough_no  \
year   zip                                               
2018.0 10001.0  1830.423318            100         1.0   
       10002.0  1640.060071            214         1.0   
       10003.0  1830.758578            172         1.0   
       10004.0  1519.703220             48         1.0   
       10005.0  1290.443401            118         1.0   
...                     ...            ...         ...   
2013.0 11436.0   220.466667            215         4.0   
       11691.0   178.396298            151         4.0   
       11692.0   201.260163             70         4.0   
       11693.0   171.277672             38         4.0   
       11694.0   256.893480            126         4.0   

                application_applications  applications  cumsum_application  \
year   zip                                                                   
2018.0 10001.0                     158.0         158.0               708.0   
       10002.0                     132.0         132.0               622.0   
       10003.0                     114.0         114.0               515.0   
       10004.0                      35.0          35.0               120.0   
       10005.0                      25.0          25.0                77.0   
...                                  ...           ...                 ...   
2013.0 11436.0                       4.0           4.0                 4.0   
       11691.0                       8.0           8.0                 8.0   
       11692.0                       1.0           1.0                 1.0   
       11693.0                       4.0           4.0                 4.0   
       11694.0                       3.0           3.0                 3.0   

                  borough         neighborhood  population  
year   zip                                                  
2018.0 10001.0  Manhattan  Chelsea and Clinton     37877.0  
       10002.0  Manhattan      Lower East Side     37877.0  
       10003.0  Manhattan      Lower East Side     37877.0  
       10004.0  Manhattan      Lower Manhattan     37877.0  
       10005.0  Manhattan      Lower Manhattan     37877.0  
...                   ...                  ...         ...  
2013.0 11436.0     Queens              Jamaica     36862.0  
       11691.0     Queens            Rockaways     36862.0  
       11692.0     Queens            Rockaways     36862.0  
       11693.0     Queens            Rockaways     36862.0  
       11694.0     Queens            Rockaways     36862.0  

[1045 rows x 9 columns]

In [83]:
# Summary statistics of the dataframe 

# Get information per year for zip codes 
# df4.query("zip == [11234, 11368]") # enter zip codes in list 

# Summary per year, aggregates zip code values for years
df4.mean(level=[0])



,ppsqft,sales_per_zip,borough_no,application_applications,applications,cumsum_application,population
year,,,,,,,
2018.0,733.970889,216.238889,2.844444,83.815642,83.815642,411.262570,47464.901734
2017.0,593.483578,160.554913,2.901734,97.393064,97.393064,337.728324,47469.864706
2016.0,537.455678,158.470930,2.889535,78.569767,78.569767,240.924419,47238.803571
2015.0,533.648233,155.160920,2.879310,78.220930,78.220930,162.895349,47022.656805
2014.0,469.058809,148.104651,2.895349,74.360465,74.360465,84.244186,47049.473373
2013.0,425.930419,139.879310,2.879310,10.259036,10.259036,10.259036,46890.029586


In [84]:
# Import shooting data

shooting_data = 'ANIQA_zip_shoot.csv'
shooting_raw = pd.read_csv(shooting_data,sep=',',low_memory=False)

shooting_raw.columns = shooting_raw.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
shooting_raw.rename(columns={'zipcode':'zip'}, inplace = True)
shooting_raw = shooting_raw.set_index(['year','zip'])
shooting_raw = shooting_raw.sort_values(by=["year","zip"])

#df = pd.DataFrame(df_raw[['ppsqft']])
#df['sales_per_zip'] = df.groupby(level=[0,1]).agg(['size'])
#df = df.mean(level=[0,1])
#df = df.sort_values(by=["year", "zip"], ascending = [False, True])

shooting = pd.DataFrame(shooting_raw[['incident_key']])
shooting['shootings_per_zip'] = shooting.groupby(level=[0,1]).agg(['size'])
shooting = shooting.drop(columns='incident_key')
shooting = shooting.mean(level=[0,1])
shooting = shooting.sort_values(by=["year","zip"], ascending=[False,True])
shooting

shootings_per_zip
year zip                     
2018 10001                  1
     10002                  2
     10012                  1
     10014                  1
     10016                  3
...                       ...
2006 11435                 11
     11436                  7
     11581                  1
     11691                 16
     11692                 17

[1580 rows x 1 columns]

In [85]:
# Look up shooting values for year and zip since there is not for every shooting year and zip code a value (perhaps no shooting in this case)

# just enter year and zip code to look up the value
a = df_raw.ix[(df_raw.index.get_level_values('year') == 2014) & (df_raw.index.get_level_values('zip') == 11375)]
a.head(40)

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated


borough               neighborhood  \
year   zip                                           
2014.0 11375.0      4.0  FLUSHING MEADOW PARK        
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                
       11375.0      4.0  FOREST HILLS                

                                     building_class_category tax_class  \
year   zip                                                               
2014.0 11375.0  07  RENTALS - WALKUP APARTMENTS                     2B   
       11375.0  01  ONE FAMILY DWELLINGS                             1   
       11375.0  01  ONE FAMILY DWELLINGS                             1   
       11375.0  01  ONE FAMILY DWELLINGS                             1   
       11375.0  01  ONE FAMILY DWELLINGS                             1   
       11375.0  01  ONE FAMILY DWELLINGS                             1   
       11375.0  01  ONE FAMILY DWELLINGS                             1   
       11375.0  01  ONE FAMILY DWELLINGS                             1   
       11375.0  01  ONE FAMILY DWELLINGS                             1   
       11375.0  01  ONE FAMILY DWELLINGS                             1   
       11375.0  01  ONE FAMILY DWELLINGS                             1   
       11375.0  01  ONE FAMILY DWELLINGS                             1   
       11375.0  01  ONE FAMILY DWELLINGS                             1   
       11375.0  01  ONE FAMILY DWELLINGS                             1   
       11375.0  01  ONE FAMILY DWELLINGS                             1   
       11375.0  01  ONE FAMILY DWELLINGS                             1   
       11375.0  01  ONE FAMILY DWELLINGS                             1   
       11375.0  01  ONE FAMILY DWELLINGS                             1   
       11375.0  01  ONE FAMILY DWELLINGS                             1   
       11375.0  01  ONE FAMILY DWELLINGS                             1   
       11375.0  01  ONE FAMILY DWELLINGS                             1   
       11375.0  01  ONE FAMILY DWELLINGS     

In [86]:
# Join shooting data to dataset

df5 = df4.join(shooting)
df5["shootings_per_zip"].fillna(0, inplace=True) # fill NaNs with zeros -> NaN means that there are no shootings in this zip code & year combination
df5

ppsqft  sales_per_zip  borough_no  \
year   zip                                               
2018.0 10001.0  1830.423318            100         1.0   
       10002.0  1640.060071            214         1.0   
       10003.0  1830.758578            172         1.0   
       10004.0  1519.703220             48         1.0   
       10005.0  1290.443401            118         1.0   
...                     ...            ...         ...   
2013.0 11436.0   220.466667            215         4.0   
       11691.0   178.396298            151         4.0   
       11692.0   201.260163             70         4.0   
       11693.0   171.277672             38         4.0   
       11694.0   256.893480            126         4.0   

                application_applications  applications  cumsum_application  \
year   zip                                                                   
2018.0 10001.0                     158.0         158.0               708.0   
       10002.0                     132.0         132.0               622.0   
       10003.0                     114.0         114.0               515.0   
       10004.0                      35.0          35.0               120.0   
       10005.0                      25.0          25.0                77.0   
...                                  ...           ...                 ...   
2013.0 11436.0                       4.0           4.0                 4.0   
       11691.0                       8.0           8.0                 8.0   
       11692.0                       1.0           1.0                 1.0   
       11693.0                       4.0           4.0                 4.0   
       11694.0                       3.0           3.0                 3.0   

                  borough         neighborhood  population  shootings_per_zip  
year   zip                                                                     
2018.0 10001.0  Manhattan  Chelsea and Clinton     37877.0                1.0  
       10002.0  Manhattan      Lower East Side     37877.0                2.0  
       10003.0  Manhattan      Lower East Side     37877.0                0.0  
       10004.0  Manhattan      Lower Manhattan     37877.0                0.0  
       10005.0  Manhattan      Lower Manhattan     37877.0                0.0  
...                   ...                  ...         ...                ...  
2013.0 11436.0     Queens              Jamaica     36862.0               11.0  
       11691.0     Queens            Rockaways     36862.0               18.0  
       11692.0     Queens            Rockaways     36862.0               16.0  
       11693.0     Queens            Rockaways     36862.0                0.0  
       11694.0     Queens            Rockaways     36862.0                0.0  

[1045 rows x 10 columns]

In [87]:
# Import csv AIRBNB data
from pandas import DataFrame

airbnb = 'dataframe_aribnb_cumsum.csv'
df_airbnb = DataFrame(pd.read_csv(airbnb,sep=',',low_memory=False))

df_airbnb = df_airbnb.set_index(['year','zip'])
df_airbnb = df_airbnb.sort_values(by=["year","zip"], ascending = [False, True])
df_airbnb = df_airbnb.drop(['counter'], axis = 1)

df_airbnb

listings_per_zip
year   zip                      
2018.0 10001.0               555
       10002.0              1316
       10003.0              1227
       10004.0                75
       10005.0               217
...                          ...
2008.0 11220.0                 1
       11221.0                 2
       11223.0                 6
       11238.0                 1
       11373.0                 2

[1458 rows x 1 columns]

In [88]:
df6 = df5.join(df_airbnb)
df6["listings_per_zip"].fillna(0, inplace=True)

In [61]:
df6

ppsqft  sales_per_zip  borough_no  \
year   zip                                               
2018.0 10001.0  1830.423318            100         1.0   
       10002.0  1640.060071            214         1.0   
       10003.0  1830.758578            172         1.0   
       10004.0  1519.703220             48         1.0   
       10005.0  1290.443401            118         1.0   
...                     ...            ...         ...   
2013.0 11436.0   220.466667            215         4.0   
       11691.0   178.396298            151         4.0   
       11692.0   201.260163             70         4.0   
       11693.0   171.277672             38         4.0   
       11694.0   256.893480            126         4.0   

                application_applications  applications  cumsum_application  \
year   zip                                                                   
2018.0 10001.0                     158.0        1580.0              7080.0   
       10002.0                     132.0        1320.0              6220.0   
       10003.0                     114.0        1140.0              5150.0   
       10004.0                      35.0         350.0              1200.0   
       10005.0                      25.0         250.0               770.0   
...                                  ...           ...                 ...   
2013.0 11436.0                       4.0          40.0                40.0   
       11691.0                       8.0          80.0                80.0   
       11692.0                       1.0          10.0                10.0   
       11693.0                       4.0          40.0                40.0   
       11694.0                       3.0          30.0                30.0   

                  borough         neighborhood  population  shootings_per_zip  \
year   zip                                                                      
2018.0 10001.0  Manhattan  Chelsea and Clinton     37877.0                1.0   
       10002.0  Manhattan      Lower East Side     37877.0                2.0   
       10003.0  Manhattan      Lower East Side     37877.0                0.0   
       10004.0  Manhattan      Lower Manhattan     37877.0                0.0   
       10005.0  Manhattan      Lower Manhattan     37877.0                0.0   
...                   ...                  ...         ...                ...   
2013.0 11436.0     Queens              Jamaica     36862.0               11.0   
       11691.0     Queens            Rockaways     36862.0               18.0   
       11692.0     Queens            Rockaways     36862.0               16.0   
       11693.0     Queens            Rockaways     36862.0                0.0   
       11694.0     Queens            Rockaways     36862.0                0.0   

                listings_per_zip  
year   zip                        
2018.0 10001.0             555.0  
       10002.0            1316.0  
       10003.0            1227.0  
       10004.0              75.0  
       10005.0             217.0  
...                          ...  
2013.0 11436.0               9.0  
       11691.0               2.0  
       11692.0              16.0  
       11693.0              12.0  
       11694.0               0.0  

[1045 rows x 11 columns]

In [89]:
#################################
## RUN ONLY WHEN YOU ACTUALLY WANT TO EXPORT DATA
# Export dataframe "df" to csv for visualization in tableau

df6.to_csv(r'dataframe_2013-2018_F.csv')